In [1]:
from PIL import Image, ImageOps
import cv2
import json
import numpy as np

import matplotlib.pyplot as plt
import random

import pillowfight

img_path = '../OCR_data/output/'

In [7]:
def PIL_img_show(img):
    Image.fromarray(img).show()

def filte_contours(contours, hierarchy):#汉字的一个特点是文字内部有空洞，由此形成连通域，但是检测句子所在行的话应该去掉这部分
    first_child = hierarchy[0][0][2]
    next_child = first_child
    all_children=[]
    while(next_child!=-1):
        all_children.append(next_child)
        next_child = hierarchy[0][next_child][0]
    contours_filtered = [contours[x] for x in all_children]
    
#     print(len(all_children), all_children)
    return contours_filtered


def get_likely_text_area(bin_img):
    
    img_height, img_width = bin_img.shape[:2]
    
    kernel_size = (int(img_height/300), int(img_width/1000))
    kernel = np.ones(kernel_size,np.uint8)
    dilate_img = cv2.dilate(bin_img,kernel,iterations = 1)
    #PIL_img_show(dilate_img)

    kernel_size = (int(img_height/400), int(img_width/80))
    kernel = np.ones(kernel_size,np.uint8)
    erosion_img = cv2.erode(dilate_img,kernel,iterations = 2)

    #PIL_img_show(erosion_img)
    
    
    image, contours, hierarchy = cv2.findContours(erosion_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#CHAIN_APPROX_SIMPLE

#     tmp_img = erosion_img-erosion_img
#     cv2.drawContours(tmp_img,contours,-1,(100,100,100),5)
#     PIL_img_show(tmp_img)

#     print(len(contours))
    contours_filtered = filte_contours(contours, hierarchy)
#    print(len(contours_filtered))
    areas=[]
    rects=[]#矩形信息包括：矩形中心，高度，宽度，旋转角
    boxs=[]#矩形信息包括：矩形四个点的坐标，经过rects信息计算而来，经过了计算把浮点数转化为图像坐标，矩形可以不是水平的。

    for cnt in contours_filtered:
        area = cv2.contourArea(cnt)
        areas.append(area)

        rect = cv2.minAreaRect(cnt) # 得到最小外接矩形的（中心(x,y), (宽,高), 旋转角度）
        rects.append(rect)

        box = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x 获取最小外接矩形的4个顶点
        box = np.int0(box)
        boxs.append(box)

    tmp_img = gray_img.copy()
    #tmp_img = erosion_img-erosion_img
    cv2.drawContours(tmp_img,contours_filtered,-1,(100,100,100),5)
    PIL_img_show(tmp_img)
    
    boxs_horizontal=[]#将boxs化为方向水平，但是如果句子是倾斜的则圈出的范围过于宽泛。
    for box in boxs:
        x1=min(box[0][0], box[1][0], box[2][0], box[3][0])
        x2=max(box[0][0], box[1][0], box[2][0], box[3][0])
        y1=min(box[0][1], box[1][1], box[2][1], box[3][1])
        y2=max(box[0][1], box[1][1], box[2][1], box[3][1])
        boxs_horizontal.append(np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]]))

#     print(len(boxs_horizontal))
    return areas, boxs_horizontal, boxs, contours_filtered

#输入：灰度图，因为要保护mask变量，需要二值化为非255的二值图。干脆封装起来避免用户输入的二值化图带有255
#输出：文字区域的面积
def get_text_area(gray_img):#使用swt算法挖掘文字区域，经过测试，该算法准确率极高
    
    img_heigth, img_width = gray_img.shape[:2]
    
    bin_threshold, bin_img = cv2.threshold(gray_img,0,100 ,cv2.THRESH_BINARY+cv2.THRESH_OTSU)#此处取100，只要不取255就行，因为mask用的是255
    
    #将文字区域用黑色矩形框画出，非文字区域用白色画出
    original_img_mask = np.uint8(np.ones((img_heigth,img_width))*255)
    
    bin_img_PIL_formate = Image.fromarray(bin_img)
    
    PIL_img_show(gray_img)
    bin_img_PIL_formate.show()
    
    swt_out_img = pillowfight.swt(bin_img_PIL_formate, output_type=pillowfight.SWT_OUTPUT_ORIGINAL_BOXES)
    
    swt_out_img.show()
    
    swt_out_img = np.array(swt_out_img)[:, :, 0]#PIL Image 格式的图像是3通道的，取其中之一即可
    #print(swt_out_img.shape)
    mask_not_text_area=np.uint8(swt_out_img!=original_img_mask)#不相等的区域是文字区域
    #print(mask_not_text_area.shape)
    
    #draw
    #Image.fromarray(255*(1-mask_not_text_area)).show()
    
    
    #mask矩阵求和即是文字区域的面积
    swt_area_sum=sum(sum(np.uint16(mask_not_text_area)))#不用uint16的话会导致第一层sum‘溢出’
    
    return swt_area_sum

def text_area_div_likely_text_area(bin_img, gray_img):
    
    areas, boxs_horizontal, boxs, contours_filtered = get_likely_text_area(bin_img)
    if areas==[]:
        return 1000000    #分母为0
    else:
        areas_sum = sum(areas)
    
    text_area_sum = get_text_area(gray_img)
    
    return text_area_sum/areas_sum
    

In [10]:
for img_id in range(6,7):
    img_name = 'img_'+str(1000000+img_id)+'.png'
    img = cv2.imread(img_path + img_name)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_heigth, img_width = gray_img.shape[:2]
    
    bin_threshold, bin_img = cv2.threshold(gray_img,0,255 ,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    
    print(img_id, text_area_div_likely_text_area(bin_img, gray_img))
    

6 0.724011518724172


In [ ]:
for img_id in range(0,42):
    img_name = 'img_'+str(1000000+img_id)+'.png'
    img = cv2.imread(img_path + img_name)
    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_heigth, img_width = gray_img.shape[:2]
    
    bin_threshold, bin_img = cv2.threshold(gray_img,0,100 ,cv2.THRESH_BINARY+cv2.THRESH_OTSU)#此处不适合取255，因为mask用的是255

    
#     PIL_img_show(erosion_img)
    areas, boxs_horizontal, boxs, contours_filtered = get_likely_text_area(bin_img)
    #text_rate_metric = get_text_rate(bin_img, boxs_horizontal)
    #print(img_id, text_rate_metric)
    
    original_img_mask = np.uint8(np.ones((img_heigth,img_width))*255)
    
    bin_img_PIL_formate = Image.fromarray(bin_img)
    #bin_img_PIL_formate.show()
    
    swt_out_img = pillowfight.swt(bin_img_PIL_formate, output_type=pillowfight.SWT_OUTPUT_ORIGINAL_BOXES)
    #swt_out_img.show()
    
    swt_out_img = np.array(swt_out_img)[:, :, 0]#PIL Image 格式的图像是3通道的，取其中之一即可
    #print(swt_out_img.shape)
    mask_not_text_area=np.uint8(swt_out_img!=original_img_mask)
    #print(mask_not_text_area.shape)
    
    #draw
    #Image.fromarray(255*(1-mask_not_text_area)).show()
    
    #draw contours
    tmp_img = bin_img-bin_img
    cv2.drawContours(tmp_img,contours_filtered,-1,(100,100,100),5)
    #PIL_img_show(tmp_img)
    
    swt_area_sum=sum(sum(np.uint16(mask_not_text_area)))
    print(img_id, '文字区域面积/连通域面积：%2.4f'%(swt_area_sum/sum(areas)))
    
#     for box_hor in boxs_horizontal:
        
#         x1,y1=box_hor[0]
#         x2,y2=box_hor[2]
#         split_img = bin_img[y1:y2, x1:x2]
#         img_x, img_y = split_img.shape[:2]
        
#         split_img_PIL_formate=Image.fromarray(split_img)
#         split_img_PIL_formate.show()
        
        
#         original_img_mask = np.uint8(np.ones((img_x,img_y))*255)
#         #SWT_OUTPUT_BW_TEXT
#         #SWT_OUTPUT_GRAYSCALE_TEXT
#         #SWT_OUTPUT_ORIGINAL_BOXES
#         swt_out_img = pillowfight.swt(split_img_PIL_formate, output_type=pillowfight.SWT_OUTPUT_ORIGINAL_BOXES)
#         swt_out_img.show()
        
#         swt_out_img = np.array(swt_out_img)[:,:,0]
#         #print(swt_out_img.shape)
#         mask_not_text_area=np.uint8(swt_out_img==original_img_mask)
#         #print(mask_not_text_area.shape)
#         Image.fromarray(255*mask_not_text_area).show()
    

In [ ]:
a=[]
for cnt in contours_filtered:
    area = cv2.contourArea(cnt)
    a.append(area)
    
a.sort()
print(a)
print(areas)

In [ ]:
print(sum(a))

In [ ]:
print(sum(sum(np.uint16(mask_not_text_area),0),1))

a=sum(mask_not_text_area)
random.choice(a)

In [ ]:
img = cv2.imread(img_path + 'img_'+str(1000017)+'.png')
gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

img_heigth, img_width = gray_img.shape[:2]
print(img_heigth, img_width)


bin_threshold, bin_img = cv2.threshold(gray_img,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
# PIL_img_show(gray_img)
PIL_img_show(bin_img)

kernel_size = (int(img_heigth/400), int(img_width/1000))
kernel = np.ones(kernel_size,np.uint8)
dilate_img = cv2.dilate(bin_img,kernel,iterations = 1)
PIL_img_show(dilate_img)

kernel_size = (int(img_heigth/400), int(img_width/80))
kernel = np.ones(kernel_size,np.uint8)
erosion_img = cv2.erode(dilate_img,kernel,iterations = 2)
PIL_img_show(erosion_img)

In [ ]:
#image,contours,hierarchy=cv2.findContours(erosion_img,1,2)

# cv2.RETR_EXTERNAL表示只检测外轮廓
# cv2.RETR_LIST检测的轮廓不建立等级关系
# cv2.RETR_CCOMP建立两个等级的轮廓，上面的一层为外边界，里面的一层为内孔的边界信息。如果内孔内还有一个连通物体，这个物体的边界也在顶层。
# cv2.RETR_TREE建立一个等级树结构的轮廓。
image, contours, hierarchy = cv2.findContours(erosion_img,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)#CHAIN_APPROX_SIMPLE

tmp_img = erosion_img-erosion_img
cv2.drawContours(tmp_img,contours,-1,(100,100,100),5)
PIL_img_show(tmp_img)

print(len(contours))
contours_filtered = contours_filter(contours, hierarchy)
print(len(contours_filtered))
areas=[]
rects=[]#矩形信息包括：矩形中心，高度，宽度，旋转角
boxs=[]#矩形信息包括：矩形四个点的坐标，经过rects信息计算而来，经过了计算把浮点数转化为图像坐标，矩形可以不是水平的。

for cnt in contours_filtered:
    area = cv2.contourArea(cnt)
    areas.append(area)
    
    rect = cv2.minAreaRect(cnt) # 得到最小外接矩形的（中心(x,y), (宽,高), 旋转角度）
    rects.append(rect)
    
    box = cv2.boxPoints(rect) # cv2.boxPoints(rect) for OpenCV 3.x 获取最小外接矩形的4个顶点
    box = np.int0(box)
    boxs.append(box)

tmp_img = erosion_img-erosion_img
cv2.drawContours(tmp_img,boxs,-1,(100,100,100),5)
PIL_img_show(tmp_img)


In [ ]:
boxs_horizontal=[]#将boxs化为方向水平，但是如果句子是倾斜的则圈出的范围过于宽泛。
for box in boxs:
    x1=min(box[0][0], box[1][0], box[2][0], box[3][0])
    x2=max(box[0][0], box[1][0], box[2][0], box[3][0])
    y1=min(box[0][1], box[1][1], box[2][1], box[3][1])
    y2=max(box[0][1], box[1][1], box[2][1], box[3][1])
    boxs_horizontal.append(np.array([[x1,y1],[x2,y1],[x2,y2],[x1,y2]]))
    
print(len(boxs_horizontal))

cv2.drawContours(gray_img,boxs_horizontal,-1,(100,100,100),5)
PIL_img_show(gray_img)

In [ ]:
small_point_cnt=0
large_not_text_cnt=0
small_not_text_cnt=0
all_text_str=[]
all_ROI_area_rate=[]
text_rate_metric=0

for idx, box_horizontal in enumerate(boxs_horizontal):
    x1,y1=box_horizontal[0]
    x2,y2=box_horizontal[2]
    img_ROI = bin_img[y1:y2, x1:x2]
    ROI_area = (x2-x1)*(y2-y1)
    ROI_area_rate = ROI_area/img_heigth/img_width
    
    
    if ROI_area/img_heigth/img_width<0.0005:#是小的噪点，不是中文或字母
        small_point_cnt+=1
    elif ((sum(sum(1-img_ROI/255)))/ROI_area)>0.7:#黑色部分太大，属于大片黑色非文字区域
        large_not_text_cnt+=1
    else:
        text_str = pytesseract.image_to_string(Image.fromarray(img_ROI), lang='chi_sim')
        if text_str=='' or text_str ==' ' or text_str=='\t':# (len(text_str)<=3 and '\t' in text_str):
            small_not_text_cnt+=1
        else:
            #print(idx, text_str, ROI_area_rate, ROI_area_rate)
            all_text_str.append(text_str)
            all_ROI_area_rate.append(ROI_area_rate)
print(small_point_cnt, large_not_text_cnt, small_not_text_cnt)

ROI_area_weights=np.array(all_ROI_area_rate/sum(all_ROI_area_rate))
all_test_length=[len(x) for x in all_text_str]
length_div_area=np.array(all_test_length)/np.array(all_ROI_area_rate)

text_rate_metric = np.mean((length_div_area-length_div_area.mean())**2*ROI_area_weights)
print(text_rate_metric)

# for i in range(len(all_text_str)):
#     text_rate_metric+=len(all_text_str[i])/all_ROI_area_rate[i]*ROI_area_weights[i]

# print(text_rate_metric/len(all_text_str))
print(all_text_str)

In [ ]:
length_div_area=np.array(all_test_length)/np.array(all_ROI_area_rate)
print(length_div_area)
print(ROI_area_weights)
print(length_div_area.mean())

In [ ]:
x=[]
for idx, box_horizontal in enumerate(boxs_horizontal):
    x1,y1=box_horizontal[0]
    x2,y2=box_horizontal[2]
    x.append((x2-x1)*(y2-y1))
print(np.sort(x)/img_heigth/img_width)

In [ ]:
print(contours[areas.index(max(areas))])
print(areas.index(max(areas)))

areas_sorted = areas.copy()
areas_sorted.sort()
areas_sorted = [x/(img_heigth*img_width)*100 for x in areas_sorted]
if areas_sorted[-1]>99.5:# to remove the largest contours if the largest is the whole image.
    print('remove the largest one if it is the whole image')
    areas_sorted = areas_sorted[:-1]
print(areas_sorted[-10:])

s=0
for x in range(img_heigth):
    for y in range(img_width):
        s+=erosion_img.item(x,y)
print(sum(areas_sorted), 1-s/img_heigth/img_width, img_heigth, img_width)

In [ ]:
cv2.aruco_CharucoBoard()
retval=cv2.aruco_CharucoBoard.getChessboardSize(bin_img)